## Module 3 - In this jupyter notebook, perfomance indicators are calculated 
* Set up: Import modules/libraries, inport data, create output folder
* Calculate efficiency (beneficial fraction)
* Calculate adquacy

to be calculated in module-6:
* Uniformity
* Relative water deficit
**=====================================================================================================================**

![title](img/Fig3_1.png)

**=====================================================================================================================**
#### <span style='background :lightgreen' > References:
* Karimi, P., Bongani, B., Blatchford, M., and de Fraiture, C.: Global satellite-based ET products for the local level irrigation management: An application of irrigation performance assessment in the sugarbelt of Swaziland, Remote Sensing, 11, 705, 2019.
* Bastiaanssen, W. G., and Bos, M.: Irrigation performance indicators based on remotely sensed data: a review of literature, Irrigation and drainage systems, 13, 291-311, 1999.
* Bastiaanssen, W. G., Van der Wal, T., and Visser, T.: Diagnosis of regional evaporation by remote sensing to support irrigation performance assessment, Irrigation and Drainage Systems, 10, 1-23, 1996.

# Set up

## i) Import packages/libraries

In [1]:
import os
master_dr = os.path.split(os.getcwd())[0]
# change the directory to where the modules are saved
os.chdir(os.path.join(os.path.split(os.getcwd())[0], "Modules"))
from GIS_functions import GIS_function as gis

## ii) Import the input data

In [4]:
# get seasonal data
seasonal_dr = os.path.join(master_dr, r"Data/tif/seasonal")
T_seasonal    = os.path.join(seasonal_dr, f"seasonal_T.tif")
AETI_seasonal    = os.path.join(seasonal_dr, f"seasonal_AETI.tif")
ETp_seasonal    = os.path.join(seasonal_dr, f"seasonal_RET.tif")
NPP_seasonal    = os.path.join(seasonal_dr, f"seasonal_NPP.tif")

# Calculate efficiency (beneficial fraction)
* Beneficial fraction is the ratio of the water that is consumed as transpiration compared to overall field water consumption (ETa). 
* $Beneficial fraction = \frac{T_a}{ET_a}$
* It is a measure of the efficiency of on farm water and agronomic practices in use of water for crop growth.

In [5]:
# collecting Geoinfo such as projection, the x and y axis
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(AETI_seasonal)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster
  
T    = gis.OpenAsArray(T_seasonal,  nan_values=True)
AETI = gis.OpenAsArray(AETI_seasonal, nan_values=True)

T_over_AETI = T/AETI

basename  = os.path.basename(AETI_seasonal).replace('AETI', 'BF')
output_fn = os.path.join(seasonal_dr, basename)
gis.CreateGeoTiff(output_fn, T_over_AETI, driver, NDV, xsize, ysize, GeoT, Projection)

/home/msalah/Desktop/m salah/WaPOR Irrigation performance assessment/application/WAPORWP_season_assessment/.venv/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


# Calculate adequacy (relative evapotranspiration)
$Adequacy= \frac{ET_a}{ET_p}$

In [6]:
## Calculate and save raster adequacy layer

# collecting Geoinfo such as projection, the x and y axis
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(AETI_seasonal)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster

AETI = gis.OpenAsArray(AETI_seasonal,  nan_values=True)
ETp  = gis.OpenAsArray(ETp_seasonal, nan_values=True)

ETa_by_ETp = AETI / ETp

# update the file name, and save into output folder
basename  = os.path.basename(AETI_seasonal).replace('AETI', 'Adequacy')
output_fn = os.path.join(seasonal_dr, basename)
gis.CreateGeoTiff(output_fn, ETa_by_ETp, driver, NDV, xsize, ysize, GeoT, Projection) 